In [6]:
import os
import psycopg2
from src.deck_scraping import *
from bs4 import BeautifulSoup

In [2]:
hostname = os.environ['CAPSTONE_DB_HOST']
dbname = os.environ['CAPSTONE_DB_DBNAME']
username = os.environ['CAPSTONE_DB_USERNAME']

In [86]:
front_page = BeautifulSoup(modern_front_page_request().content, 'html.parser')

event_ids = get_event_ids(front_page)

event_page = BeautifulSoup(event_request(event_ids[0]).content, 'html.parser')

deck_ids = get_deck_ids(event_page)

raw_deck_list = deck_request(deck_ids[0])

deck_list = format_deck(raw_deck_list)

user_card_counts = make_user_card_counts(deck_ids[0], deck_list)

user_card_counts

[(312605, 'Unclaimed Territory', 4),
 (312605, 'Seachrome Coast', 2),
 (312605, 'Plains', 1),
 (312605, 'Horizon Canopy', 4),
 (312605, 'Cavern of Souls', 4),
 (312605, 'Ancient Ziggurat', 4),
 (312605, 'Aether Vial', 4),
 (312605, "Thalia's Lieutenant", 4),
 (312605, 'Thalia, Guardian of Thraben', 4),
 (312605, 'Reflector Mage', 4),
 (312605, 'Phantasmal Image', 4),
 (312605, 'Noble Hierarch', 4),
 (312605, 'Meddling Mage', 4),
 (312605, 'Mantis Rider', 4),
 (312605, 'Kitesail Freebooter', 4),
 (312605, 'Kessig Malcontents', 1),
 (312605, 'Champion of the Parish', 4)]

In [94]:
items = ["(, '{}', {})".format(deck_id, card_name, card_count) for deck_id, card_name, card_count in user_card_counts]

items

["(312605, 'Unclaimed Territory', 4)",
 "(312605, 'Seachrome Coast', 2)",
 "(312605, 'Plains', 1)",
 "(312605, 'Horizon Canopy', 4)",
 "(312605, 'Cavern of Souls', 4)",
 "(312605, 'Ancient Ziggurat', 4)",
 "(312605, 'Aether Vial', 4)",
 "(312605, 'Thalia's Lieutenant', 4)",
 "(312605, 'Thalia, Guardian of Thraben', 4)",
 "(312605, 'Reflector Mage', 4)",
 "(312605, 'Phantasmal Image', 4)",
 "(312605, 'Noble Hierarch', 4)",
 "(312605, 'Meddling Mage', 4)",
 "(312605, 'Mantis Rider', 4)",
 "(312605, 'Kitesail Freebooter', 4)",
 "(312605, 'Kessig Malcontents', 1)",
 "(312605, 'Champion of the Parish', 4)"]

In [95]:
joined_items = ', '.join(items)

joined_items

"(312605, 'Unclaimed Territory', 4), (312605, 'Seachrome Coast', 2), (312605, 'Plains', 1), (312605, 'Horizon Canopy', 4), (312605, 'Cavern of Souls', 4), (312605, 'Ancient Ziggurat', 4), (312605, 'Aether Vial', 4), (312605, 'Thalia's Lieutenant', 4), (312605, 'Thalia, Guardian of Thraben', 4), (312605, 'Reflector Mage', 4), (312605, 'Phantasmal Image', 4), (312605, 'Noble Hierarch', 4), (312605, 'Meddling Mage', 4), (312605, 'Mantis Rider', 4), (312605, 'Kitesail Freebooter', 4), (312605, 'Kessig Malcontents', 1), (312605, 'Champion of the Parish', 4)"

In [96]:
query = '''INSERT INTO user_card_counts(deck_id, card_name, card_count)
           VALUES {}'''.format(joined_items)

query

"INSERT INTO user_card_counts(deck_id, card_name, card_count)\n           VALUES (312605, 'Unclaimed Territory', 4), (312605, 'Seachrome Coast', 2), (312605, 'Plains', 1), (312605, 'Horizon Canopy', 4), (312605, 'Cavern of Souls', 4), (312605, 'Ancient Ziggurat', 4), (312605, 'Aether Vial', 4), (312605, 'Thalia's Lieutenant', 4), (312605, 'Thalia, Guardian of Thraben', 4), (312605, 'Reflector Mage', 4), (312605, 'Phantasmal Image', 4), (312605, 'Noble Hierarch', 4), (312605, 'Meddling Mage', 4), (312605, 'Mantis Rider', 4), (312605, 'Kitesail Freebooter', 4), (312605, 'Kessig Malcontents', 1), (312605, 'Champion of the Parish', 4)"

In [135]:
conn = psycopg2.connect('dbname={} host={} user={}'.format(dbname, hostname, username))

In [136]:
cursor = conn.cursor()

In [99]:
cursor.execute(query, )

ProgrammingError: syntax error at or near "s"
LINE 2: ... 4), (312605, 'Aether Vial', 4), (312605, 'Thalia's Lieutena...
                                                             ^


In [109]:
query = '''INSERT INTO user_card_counts(deck_id, card_name, card_count)
           VALUES ({},'{}',{})'''.format(user_card_counts[0][0], user_card_counts[0][1], user_card_counts[0][2])


In [115]:
query

"INSERT INTO user_card_counts(deck_id, card_name, card_count)\n           VALUES (312605,'Unclaimed Territory',4)"

In [120]:
query = 'INSERT INTO user_card_counts(deck_id, card_name, card_count) VALUES (%s, %s, %s)'

cursor.execute(query, vars=user_card_counts[7])

In [121]:
conn.commit()

In [132]:
template = ', '.join(['%s'] * len(user_card_counts))
template

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

In [133]:
query = 'INSERT INTO user_card_counts (deck_id, card_name, card_count) VALUES {}'.format(template)

query

'INSERT INTO user_card_counts (deck_id, card_name, card_count) VALUES %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

In [137]:
cursor.execute(query, vars=user_card_counts)

In [138]:
conn.commit()